In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
file_name = "df_count_3_part2.csv"   

In [3]:
 
df = pd.read_csv(file_name , parse_dates=["date"])


C:\Users\oussa\AppData\Local\Temp\ipykernel_14324\1571972478.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name , parse_dates=["date"])


In [4]:
df.shape

(12287458, 10)

In [5]:
# Read the country reference file
df_countries = pd.read_csv("CT_COUNTRIES.csv", sep=";", header=None, names=["code", "lang", "name"])

country_dict = df_countries.set_index("code")["name"].to_dict()

In [6]:
# Define columns to ignore when checking duplicates
cols_ignore = ["RECPTCL_FID", "duration_to_next_step", "total_duration", "id"]
cols_check  = [c for c in df.columns if c not in cols_ignore]

# 1. Identify all duplicate rows based on cols_check
dup_mask = df.duplicated(subset=cols_check, keep=False)

# 2. Number of such duplicate rows
num_duplicates = dup_mask.sum()
print(f"Number of duplicate rows (ignoring specified columns): {num_duplicates}")




Number of duplicate rows (ignoring specified columns): 60126


In [7]:
df = df.drop_duplicates(subset=cols_check, keep="first").reset_index(drop=True)

In [8]:
import pandas as pd

ID_COL    = "MAILITM_FID"
EVENT_COL = "EVENT_TYPE_NM"

# 1) pick the first row per parcel (rows are already in correct order)
first_events = (
    df.groupby(ID_COL, sort=False)[EVENT_COL]
      .first()                 # a Series: index = parcel ID, value = first event
)

# 2) count how many parcels have each first event
event_counts = (
    first_events.value_counts()   # Series: index = event, value = count
               .sort_values(ascending=False)
)

# 3) print the result
print("First-event frequencies:")
for event, count in event_counts.items():
    print(f"{event:40} {count:,}")


First-event frequencies:
Réception d'envoi du client (Srt)        776,570
Expédition d'envoi à l'étranger (EDI-reçu) 261,808
Insérer envoi dans sac (Srt)             20,111


In [9]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

RECPTCL_FID                  1782961
établissement_postal         1026559
next_établissement_postal    7835253
duration_to_next_step        3167344
dtype: int64


In [10]:
df = df.drop(columns=["RECPTCL_FID", "duration_to_next_step", "total_duration"])

In [11]:
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

établissement_postal         1026559
next_établissement_postal    7835253
dtype: int64


In [12]:

# Assuming 'df' is your DataFrame!

# Identify the rows with missing values
missing_rows = df[df.isnull().any(axis=1)]

# Print the total number of rows that have missing values
print("Total rows with missing values:", missing_rows.shape[0])




Total rows with missing values: 8191232


In [13]:
# Analyze the missing patterns across rows:
# Create a binary pattern for each row where 1 indicates a missing value and 0 indicates non-missing.
missing_pattern = missing_rows.isnull().astype(int)
# Convert each pattern to a tuple to use as a key for groupby
missing_pattern_tuples = missing_pattern.apply(tuple, axis=1)
# Group by the missing pattern and count the number of rows for each pattern
pattern_summary = missing_rows.groupby(missing_pattern_tuples).size().sort_values(ascending=False)

print("\nSummary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):")
print(pattern_summary)




Summary of missing patterns (tuple of 0's and 1's corresponding to missing values in each column):
(0, 0, 0, 0, 0, 1, 0)    7164673
(0, 0, 0, 1, 0, 1, 0)     670580
(0, 0, 0, 1, 0, 0, 0)     355979
dtype: int64


In [14]:
distinct_values_count = df.nunique()

distinct_values_count = distinct_values_count.sort_values(ascending=True)

print(distinct_values_count)

EVENT_TYPE_NM                     23
EVENT_TYPE_CD                     23
id                                74
next_établissement_postal       3541
établissement_postal            3670
MAILITM_FID                  1058489
date                         9122376
dtype: int64


In [15]:

# Count the occurrences of each protocol
event_type = df['EVENT_TYPE_NM'].value_counts()

print(event_type)


EVENT_TYPE_NM
Recevoir envoi au bureau de livraison (Ent)                    2303342
Expédier envoi à adresse nationale (Ent)                       1694707
Expédier envoi à adresse nationale (Srt)                       1534491
Expédition d'envoi à l'étranger (EDI-reçu)                     1165369
Recevoir envoi au bureau d'échange (Ent)                       1097909
Vaine tentative de livraison d'envoi (Ent)                     1021503
Réception d'envoi du client (Srt)                               982430
Livraison d'envoi (Ent)                                         935019
Insérer envoi dans sac (Srt)                                    851000
Transmettre envoi à l'agent de livraison (Ent)                  614515
Enregistrer détails d'envoi au bureau d'échange (Srt)            29164
Recevoir envoi au bureau d'échange (Srt)                         20553
Expédier envoi à la douane (Ent)                                  4967
Renvoyer envoi de la douane (Ent)                              

In [16]:

# Count the occurrences of each protocol
proto_counts = df['établissement_postal'].value_counts()

print(proto_counts)


établissement_postal
ALGER COLIS POSTAUX    1445569
ALGER GARE             1308844
TCHÈQUE, RÉPUBLIQUE     737899
ÉMIRATS ARABES UNIS     483780
MALAISIE                362529
                        ...   
ALGÉRIE                      1
EL-AZLA                      1
KOLEA HAY BILLEL             1
CHLEF REBUT                  1
SÉNÉGAL                      1
Name: count, Length: 3670, dtype: int64


In [17]:
# Count the occurrences of each protocol
proto_counts = df['next_établissement_postal'].value_counts()

print(proto_counts)



next_établissement_postal
ALGÉRIE                      1165369
ALGER GARE                    422850
CONSTANTINE COLIS POSTAUX     175879
ORAN COLIS POSTAUX            175422
ANNABA EL MARSA               122806
                              ...   
AIN-ISDIEL                         1
OUM-EL-BOUAGHI-EPI-1               1
BICHER                             1
EL-KHARROUB                        1
HASSI-LABIADH                      1
Name: count, Length: 3541, dtype: int64


In [18]:

# Count the occurrences of each protocol
proto_counts = df['id'].value_counts()

print(proto_counts)

s = proto_counts


id
CZ    5391597
AE    2987896
NL    2295054
MY    1420865
US      56296
       ...   
AZ          8
HR          8
LT          6
ME          5
TJ          3
Name: count, Length: 74, dtype: int64


In [19]:
# Map the Series index using the country_dict
s.index = s.index.map(lambda code: country_dict.get(code, code))  # Keep unmapped codes as-is
print(s)

id
TCHÈQUE, RÉPUBLIQUE    5391597
ÉMIRATS ARABES UNIS    2987896
PAYS-BAS               2295054
MALAISIE               1420865
ÉTATS-UNIS               56296
                        ...   
AZERBAÏDJAN                  8
CROATIE                      8
LITUANIE                     6
ME                           5
TADJIKISTAN                  3
Name: count, Length: 74, dtype: int64


In [20]:
print("Min date:", df["date"].min())
print("Max date:", df["date"].max())

Min date: 2018-11-15 10:24:59.583
Max date: 2025-05-21 16:13:21.467


In [21]:
df["date"] = df["date"].str.replace(r'\.\d+$', '', regex=True)
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)

C:\Users\oussa\AppData\Local\Temp\ipykernel_14324\579475394.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True)


In [22]:
df = df.sort_values(by=["MAILITM_FID", "date"])

# Calculate time to next step
df["duration_to_next_step"] = df.groupby("MAILITM_FID")["date"].shift(-1) - df["date"]

# Calculate total duration per ID
first_date = df.groupby("MAILITM_FID")["date"].transform("first")
last_date = df.groupby("MAILITM_FID")["date"].transform("last")
df["total_duration"] = last_date - first_date




In [23]:
# Preview the updated dataset
df[["MAILITM_FID", "date", "EVENT_TYPE_NM", "duration_to_next_step", "total_duration"]].head(10)

,MAILITM_FID,date,EVENT_TYPE_NM,duration_to_next_step,total_duration
2940489,CB167614873US,2024-07-16 08:15:00,Expédition d'envoi à l'étranger (EDI-reçu),14 days 00:41:23,25 days 06:31:49
12195509,CB167614873US,2024-07-30 08:56:23,Recevoir envoi au bureau d'échange (Ent),0 days 02:25:21,25 days 06:31:49
12195510,CB167614873US,2024-07-30 11:21:44,Expédier envoi à adresse nationale (Ent),0 days 00:17:49,25 days 06:31:49
12195511,CB167614873US,2024-07-30 11:39:33,Expédier envoi à adresse nationale (Ent),3 days 23:17:01,25 days 06:31:49
12195512,CB167614873US,2024-08-03 10:56:34,Recevoir envoi au bureau de livraison (Ent),0 days 00:02:05,25 days 06:31:49
12195513,CB167614873US,2024-08-03 10:58:39,Enregistrer détails d'envoi au bureau d'échang...,0 days 00:09:19,25 days 06:31:49
12195514,CB167614873US,2024-08-03 11:07:58,Expédier envoi à adresse nationale (Srt),0 days 21:26:13,25 days 06:31:49
12195515,CB167614873US,2024-08-04 08:34:11,Recevoir envoi au bureau de livraison (Ent),0 days 00:03:11,25 days 06:31:49
12195516,CB167614873US,2024-08-04 08:37:22,Expédier envoi à adresse nationale (Srt),1 days 01:01:25,25 days 06:31:49
12195517,CB167614873US,2024-08-05 09:38:47,Recevoir envoi au bureau de livraison (Ent),0 days 00:08:51,25 days 06:31:49


In [26]:
# Filter rows where total_duration is NOT zero
zero_duration_df = df[df["total_duration"] <= pd.Timedelta(days = 1)]

# Display the first few rows as a check
zero_duration_df.shape


(0, 9)

In [38]:

# Count the occurrences of each protocol
countries = zero_duration_df['id'].value_counts()

print(countries)


Series([], Name: count, dtype: int64)


In [39]:
# Map the Series index using the country_dict
countries.index = countries.index.map(lambda code: country_dict.get(code, code))  
print(countries)

Series([], Name: count, dtype: int64)


In [40]:
# Step 1: Count how many times each ID appears
id_counts = zero_duration_df["MAILITM_FID"].value_counts()

# Step 2: Count how many IDs have a specific count
count_of_counts = id_counts.value_counts().sort_index()

print(count_of_counts)


Series([], Name: count, dtype: int64)


In [41]:
ids_with_0 = id_counts[id_counts == 2].index
df_with_0 = df[df["MAILITM_FID"].isin(ids_with_0)]
df_with_0.head()

,MAILITM_FID,EVENT_TYPE_NM,date,établissement_postal,EVENT_TYPE_CD,next_établissement_postal,id,duration_to_next_step,total_duration


In [42]:

# Count the occurrences of each protocol
event_type = df_with_0['EVENT_TYPE_NM'].value_counts()

print(event_type)



Series([], Name: count, dtype: int64)


In [27]:
# Drop rows where total_duration is zero
df = df[df["total_duration"] != pd.Timedelta(0)]

# Check the shape after dropping
print(df.shape)


(12257386, 9)


In [28]:
df.to_csv(file_name, index=False, encoding='utf-8-sig')